# REER weights

Computes the weights matrices for real effective exchange rates following the methodologies of Bems and Johnson (2017) and Patel, Wang, and Wei (2019). 

## Set up

In [1]:
import numpy as np
import pandas as pd
import duckdb

### Select MRIO version

In [2]:
input, output = 'adb-mrio.parquet', 'reer-weights'
# input, output = 'adb-mrio62.parquet', 'reer62-weights'

### Parameters

In [3]:
sectors = pd.read_excel('../dicts/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Compute weights matrices

In [4]:
def to_df(matrix, year, method=None):
    df = pd.DataFrame(matrix)
    df.insert(0, 't', int(year))
    if method is not None:
        df.insert(0, 'method', method)
    return df

In [5]:
DF, DF_sector = pd.DataFrame(), pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    y = np.sum(Y, axis=0)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    B = Dx @ Z
    Leontief = np.linalg.inv(np.eye(G*N) - A)
    Ghosh = np.linalg.inv(np.eye(G*N) - B)
    
    agg = np.kron(np.eye(G), np.ones((N, 1)))
    Z_agg = np.transpose(agg) @ Z @ agg
    Y_agg = np.transpose(agg) @ Y
    va_agg = va @ agg
    D_agg = Z_agg + Y_agg

    z_agg = np.sum(Z_agg, axis=0)
    d_agg = np.sum(D_agg, axis=0)
    x_agg = z_agg + va_agg
    e_agg = z_agg + y

    A_agg = Z_agg @ np.diag(1/x_agg)
    B_agg = np.diag(1/x_agg) @ Z_agg
    Leontief_agg = np.linalg.inv(np.eye(G) - A_agg)
    Ghosh_agg = np.linalg.inv(np.eye(G) - B_agg)

    # Weights matrices: Conventional

    S = np.diag(1/d_agg) @ np.transpose(D_agg)
    W = np.diag(1/e_agg) @ D_agg
    SW0 = S @ W * (1 - np.eye(G))
    Weights = np.eye(G) - np.diag(1/np.sum(SW0, axis=1)) @ SW0

    # Weights matrices: Bems & Johnson

    Sy = np.diag(1/x_agg) @ Y_agg
    Sz = np.diag(1/x_agg) @ Z_agg
    Wy = np.diag(1/y) @ np.transpose(Y_agg)
    Wz = np.diag(1/z_agg) @ np.transpose(Z_agg)
    v_agg = np.diag(va_agg/x_agg)
    SW0_BJ = Ghosh_agg @ Sy @ Wy @ np.transpose(Leontief_agg) @ v_agg * (1-np.eye(G))
    Weights_BJ = np.eye(G) - np.diag(1/np.sum(SW0_BJ, axis=1)) @ SW0_BJ

    # Weights matrices: Patel, Wang, & Wei

    SSy = Dx @ Y
    WWy = np.diag(1/y) @ np.transpose(Y)
    SW0_PWW = Ghosh @ SSy @ WWy @ np.transpose(Leontief) @ np.diag(v) * (1 - np.eye(G*N))
    Shares_PWW = np.diag(1/np.sum(SW0_PWW, axis=1))
    Weights_PWW = np.eye(G*N) - Shares_PWW @ SW0_PWW
    Weights_PWW[np.isnan(Weights_PWW)] = 0

    R = np.diag(1/va_agg) @ np.kron(np.ones((G, 1)), va) * np.transpose(np.kron(np.eye(G), np.ones((N, 1))))
    SW0_PWW_agg = R @ (Ghosh @ SSy @ WWy @ np.transpose(Leontief) @ np.diag(v)) @ agg * (1 - np.eye(G))
    Shares_PWW_agg = np.diag(1/np.sum(SW0_PWW_agg, axis=1))
    Weights_PWW_agg = np.eye(G) - Shares_PWW_agg @ SW0_PWW_agg

    # Append
    
    DF = pd.concat([
            DF,
            to_df(Weights, year, 'conventional'), 
            to_df(Weights_BJ, year, 'bj'), 
            to_df(Weights_PWW_agg, year, 'pww')
        ],
        ignore_index=True
    )
    DF_sector = pd.concat([DF_sector, to_df(Weights_PWW, year)], ignore_index=True)

    print(f'{year} done')

DF.to_parquet(f'../data/reer/{output}.parquet', index=False)
DF_sector.to_parquet(f'../data/reer/{output}-sector.parquet', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


### View results

In [6]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/reer/{output}.parquet')").df()

,method,t,0,1,2,3,4,5,6,7,...,63,64,65,66,67,68,69,70,71,72
0,conventional,2017,1.000000,-0.002230,-0.006339,-0.000593,-0.005609,-0.007088,-0.007245,-0.213935,...,-0.000421,-0.000168,-0.000043,-0.000075,-0.001138,-0.001517,-0.003949,-0.005473,-0.028782,-0.191865
1,conventional,2017,-0.003783,1.000000,-0.013728,-0.003940,-0.003744,-0.005797,-0.031935,-0.031749,...,-0.000677,-0.000189,-0.000126,-0.000288,-0.001173,-0.000745,-0.003249,-0.003886,-0.000848,-0.210596
2,conventional,2017,-0.005380,-0.006866,1.000000,-0.002697,-0.007135,-0.006258,-0.018420,-0.034197,...,-0.001010,-0.000393,-0.000132,-0.000176,-0.001546,-0.000612,-0.007796,-0.007132,-0.000917,-0.196201
3,conventional,2017,-0.005343,-0.020913,-0.028619,1.000000,-0.004653,-0.004981,-0.009002,-0.034276,...,-0.000473,-0.000276,-0.003898,-0.002977,-0.003038,-0.000555,-0.003188,-0.002956,-0.000624,-0.206147
4,conventional,2017,-0.006799,-0.002675,-0.010191,-0.000626,1.000000,-0.011306,-0.005201,-0.134588,...,-0.008900,-0.001930,-0.000131,-0.000468,-0.005164,-0.000959,-0.008533,-0.003988,-0.000698,-0.211112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,pww,2022,-0.008113,-0.003007,-0.005324,-0.000921,-0.009692,-0.008735,-0.009691,-0.101203,...,-0.001850,-0.000700,-0.000358,-0.000343,-0.004413,1.000000,-0.028260,-0.022772,-0.001436,-0.364692
1310,pww,2022,-0.010457,-0.003679,-0.006010,-0.000943,-0.011956,-0.009829,-0.011251,-0.176341,...,-0.001822,-0.000698,-0.000310,-0.000304,-0.013192,-0.006645,1.000000,-0.025016,-0.001851,-0.197601
1311,pww,2022,-0.012202,-0.004175,-0.005970,-0.000822,-0.013798,-0.010326,-0.012935,-0.140448,...,-0.001786,-0.000727,-0.000523,-0.000427,-0.005249,-0.008486,-0.039647,1.000000,-0.002671,-0.171315
1312,pww,2022,-0.090191,-0.003407,-0.005133,-0.000431,-0.006232,-0.014670,-0.011827,-0.238940,...,-0.001368,-0.000630,-0.000114,-0.000202,-0.002408,-0.002001,-0.010978,-0.009994,1.000000,-0.077696


In [7]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/reer/{output}-sector.parquet')").df()

,t,0,1,2,3,4,5,6,7,8,...,2545,2546,2547,2548,2549,2550,2551,2552,2553,2554
0,2017,1.000000,-0.009254,-0.007464,-0.000539,-0.000073,-0.001084,-0.003873,-0.001526,-0.002223,...,-0.000526,-0.001023,-0.002670,-0.002394,-0.003167,-0.001981,-0.001536,-0.000924,-0.001127,-0.000164
1,2017,-0.002942,1.000000,-0.001754,-0.000130,-0.000018,-0.000255,-0.000882,-0.000365,-0.000517,...,-0.000606,-0.001918,-0.004107,-0.004911,-0.004741,-0.004319,-0.003255,-0.001791,-0.002142,-0.000351
2,2017,-0.012918,-0.009549,1.000000,-0.000560,-0.000076,-0.001125,-0.004018,-0.001583,-0.002308,...,-0.000471,-0.000730,-0.002173,-0.001577,-0.002649,-0.001241,-0.000976,-0.000625,-0.000821,-0.000117
3,2017,-0.010568,-0.007992,-0.006346,1.000000,-0.000063,-0.000919,-0.003289,-0.001298,-0.001896,...,-0.000664,-0.001766,-0.003980,-0.004545,-0.004529,-0.003934,-0.002973,-0.001699,-0.001898,-0.000291
4,2017,-0.008785,-0.006806,-0.005270,-0.000386,1.000000,-0.000764,-0.002732,-0.001080,-0.001573,...,-0.000676,-0.001815,-0.004131,-0.004674,-0.004658,-0.004061,-0.003061,-0.001743,-0.001950,-0.000299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15325,2022,-0.000150,-0.001719,-0.000052,-0.000004,-0.000006,-0.000011,-0.000026,-0.000021,-0.000027,...,-0.005550,-0.015341,-0.040202,-0.048827,-0.041130,1.000000,-0.032463,-0.020466,-0.017096,-0.001966
15326,2022,-0.000145,-0.001686,-0.000049,-0.000004,-0.000006,-0.000011,-0.000025,-0.000021,-0.000026,...,-0.005568,-0.015399,-0.040343,-0.049021,-0.041269,-0.044644,1.000000,-0.020548,-0.017161,-0.001974
15327,2022,-0.000146,-0.001654,-0.000050,-0.000004,-0.000006,-0.000011,-0.000025,-0.000020,-0.000027,...,-0.005450,-0.015070,-0.039488,-0.047970,-0.040393,-0.043687,-0.031894,1.000000,-0.016794,-0.001932
15328,2022,-0.000180,-0.001778,-0.000064,-0.000005,-0.000005,-0.000013,-0.000030,-0.000020,-0.000032,...,-0.004820,-0.013244,-0.034813,-0.042102,-0.035653,-0.038327,-0.027975,-0.017639,1.000000,-0.001694
